## Overview

Repo: https://github.com/pgahq/instructor-groq-openai-llm-examples

This notebook shows how to use Instructor with LLMs to generate simple Python code. It uses validation to ensure the dependencies are correct and that the code runs without errors.

Note: this notebook assumes you're using Google Colab. You can safely edit / play here. Or go to `File` -> `Save a copy in Google Drive` to make your own version.

In [1]:
!pip install --quiet instructor groq openai jsonref


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


On the left, click the key and set two secrets with your keys. Be sure to enable "Notebook access" for them. This is how Google Colab works...you're not sharing your keys with anyone.

OPENAI_API_KEY - get a key from https://platform.openai.com/api-keys

GROQ_API_KEY - get a key from https://console.groq.com/keys

In [2]:
import instructor
import openai
import groq
from pydantic import BaseModel, Field, field_validator
from typing import List
import subprocess
from rich import print as rprint
import os

try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = '' or userdata.get('OPENAI_API_KEY') # or put your key in the '' on this line
    os.environ['GROQ_API_KEY'] = '' or userdata.get('GROQ_API_KEY')
except Exception as e:
    # print(e)
    pass

if not os.environ.get('OPENAI_API_KEY') or not os.environ.get('GROQ_API_KEY'):
    raise ValueError("Both OPENAI_API_KEY and GROQ_API_KEY environment variables must be set and non-empty. Read the text in the notebook (above this block) for more info.")

Now to the cool stuff...

In [5]:
inference_provider = "groq"   # "openai" or "groq"
client = instructor.from_openai(openai.OpenAI()) if inference_provider == "openai" else instructor.from_groq(groq.Groq())

class CodeFile(BaseModel):
    """
    Python code and dependencies to meet specified requirements. Dependencies will be installed and the code will be executed.

    Limitations:
    - This code will not be tested for quality, only that it runs without errors.
    """
    architecture: str = Field(description="A markdown file that describes the architecture of the code and how it will satisfy each requirement one by one.")
    dependencies: List[str] = Field(description="List of pip packages required to run the code.")
    code: str = Field(description="A single Python file.")

    # the order above specifies the order of validation

    @field_validator('dependencies')
    @classmethod
    def validate_dependencies(cls, v):
        if v:
            try:
                subprocess.run(["pip", "install"] + v, check=True)
                print("Dependencies installed: " + ", ".join(v))
                print("\n\n")
            except subprocess.CalledProcessError as e:
                error_value = "Failed to install dependencies: " + str(e)
                print(error_value)
                raise ValueError(error_value)
        return v

    @field_validator('code')
    @classmethod
    def validate_code(cls, v):
        if v:
            try:
                print("Python code\n===========\n")
                rprint(v)
                print("\n\nTrying code\n===========\n")
                exec(v)
            except Exception as e:
                error_value = "Error while executing code: " + str(e)
                print(error_value)
                raise ValueError(error_value)
        return v

result = client.chat.completions.create(
    model="llama-3.1-70b-versatile" if inference_provider == "groq" else "gpt-4o",
    response_model=CodeFile,
    temperature=0.0,
    max_retries=3,
    messages=[{"role": "user", "content": """
        Extract the text content of https://example.com as markdown and print it out. Be thoughtful to only extract the content as a human user would want it rather than the HTML and other source code.
        """}]
    )

code = result.code
# rprint(code)

Dependencies installed: requests, beautifulsoup4



Python code




[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


import requests
from bs4 import BeautifulSoup

url = "https://example.com"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
print(soup.get_text())



Trying code




Example Domain







Example Domain
This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.
More information...




